<a href="https://colab.research.google.com/github/wlg100/numseqcont_circuit_expms/blob/main/notebook_templates/headFNs_expms_template.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" align="left"/></a>&nbsp;or in a local notebook.

# Setup

In [1]:
# %pip install git+https://github.com/neelnanda-io/TransformerLens.git

In [2]:
# import torch
# import transformer_lens
# import transformer_lens.utils as utils
# from transformer_lens.hook_points import (
#     HookedRootModule,
#     HookPoint,
# )  # Hooking utilities
# from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
# torch.set_grad_enabled(False)

In [3]:
# model_name = "gpt2-small"
# model = HookedTransformer.from_pretrained(
#     model_name,
#     center_unembed=True,
#     center_writing_weights=True,
#     fold_ln=True,
#     refactor_factored_attn_matrices=True,
# )

In [4]:
%%capture
%pip install git+https://github.com/redwoodresearch/Easy-Transformer.git
%pip install einops datasets transformers fancy_einsum

In [6]:
# assert torch.cuda.device_count() == 1
# from tqdm import tqdm
# import pandas as pd
import torch
import torch as t
from easy_transformer.EasyTransformer import (
    EasyTransformer,
)
# from time import ctime
# from functools import partial

import numpy as np
import einops
from copy import deepcopy

In [7]:
model = EasyTransformer.from_pretrained("gpt2").cuda()
# model = EasyTransformer.from_pretrained("gpt2")
model.set_use_attn_result(True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/easy_transformer/components.py:616: UserWarning: Moved LN1 to the attention block
  warnings.warn("Moved LN1 to the attention block")


Moving model to device:  cuda
Finished loading pretrained model gpt2 into EasyTransformer!


# Dataset and Score Functions

In [20]:
class Dataset:
    def __init__(self, prompts, tokenizer, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if key != 'text']:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                    target_token = prompt[targ]
                else:
                    target_token = "Ġ" + prompt[targ]
                target_index = tokens.index(target_token)
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

In [22]:
def get_next_scores(model, layer, head, dataset, task="numerals", neg=False, print_all_results=True):
    cache = {}
    model.cache_some(cache, lambda x: x == "blocks.0.hook_resid_post")
    model(dataset.toks.long())
    if neg:
        sign = -1
    else:
        sign = 1
    z_0 = model.blocks[1].attn.ln1(cache["blocks.0.hook_resid_post"])
    # z_0 = model.blocks[1].attn.hook_z(cache["blocks.0.hook_resid_post"])

    v = torch.einsum("eab,bc->eac", z_0, model.blocks[layer].attn.W_V[head])
    v += model.blocks[layer].attn.b_V[head].unsqueeze(0).unsqueeze(0)

    o = sign * torch.einsum("sph,hd->spd", v, model.blocks[layer].attn.W_O[head])
    logits = model.unembed(model.ln_final(o))

    k = 5
    n_right = 0

    pred_tokens_dict = {}
    words_moved = []
    # get the keys from the first prompt in the dataset
    words = [key for key in dataset.prompts[0].keys() if key != 'text']

    numwords = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    ranks = ['first', 'second', 'third', 'fourth', 'fifth', 'sixth', 'seventh', 'eighth', 'ninth', 'tenth', 'eleventh', 'twelfth']

    for seq_idx, prompt in enumerate(dataset.prompts):
        for word in words:
            pred_tokens = [
                model.tokenizer.decode(token)
                for token in torch.topk(
                    logits[seq_idx, dataset.word_idx[word][seq_idx]], k
                ).indices
            ]

            # get next member after seq member prompt[word]
            if task == "numerals":
                next_word = str(int(prompt[word]) + 1)
            elif task == "numwords":
                next_word = str(numwords[numwords.index(prompt[word]) + 1])
            elif task == "months":
                next_word = str(ranks[months.index(prompt[word]) + 1])

            nextToken_in_topK = 'no'
            if " " + next_word in pred_tokens or next_word in pred_tokens:
                n_right += 1
                words_moved.append(prompt[word])
                nextToken_in_topK = 'yes'
            pred_tokens_dict[prompt[word]] = (pred_tokens, next_word, nextToken_in_topK)

    percent_right = (n_right / (dataset.N * len(words))) * 100
    if percent_right > 0:
        print(f"Next circuit for head {layer}.{head}: Top {k} accuracy: {percent_right}%")

    if print_all_results == True:
        print(pred_tokens_dict)

    return percent_right

In [23]:
def get_copy_scores(model, layer, head, dataset, neg=False, print_all_results=True):
    cache = {}
    model.cache_some(cache, lambda x: x == "blocks.0.hook_resid_post")
    model(dataset.toks.long())
    if neg:
        sign = -1
    else:
        sign = 1
    z_0 = model.blocks[1].attn.ln1(cache["blocks.0.hook_resid_post"])
    # z_0 = model.blocks[1].attn.hook_z(cache["blocks.0.hook_resid_post"])

    v = torch.einsum("eab,bc->eac", z_0, model.blocks[layer].attn.W_V[head])
    v += model.blocks[layer].attn.b_V[head].unsqueeze(0).unsqueeze(0)

    o = sign * torch.einsum("sph,hd->spd", v, model.blocks[layer].attn.W_O[head])
    logits = model.unembed(model.ln_final(o))

    k = 5
    n_right = 0

    pred_tokens_dict = {}
    words_moved = []
    # get the keys from the first prompt in the dataset
    words = [key for key in dataset.prompts[0].keys() if key != 'text']

    for seq_idx, prompt in enumerate(dataset.prompts):
        for word in words:
            pred_tokens = [
                model.tokenizer.decode(token)
                for token in torch.topk(
                    logits[seq_idx, dataset.word_idx[word][seq_idx]], k
                ).indices
            ]

            token_in_topK = 'no'
            if " " + prompt[word] in pred_tokens or prompt[word] in pred_tokens:
                n_right += 1
                words_moved.append(prompt[word])
                token_in_topK = 'yes'
            pred_tokens_dict[prompt[word]] = (pred_tokens, token_in_topK)

    percent_right = (n_right / (dataset.N * len(words))) * 100
    if percent_right > 0:
        print(f"Copy circuit for head {layer}.{head}: Top {k} accuracy: {percent_right}%")

    if print_all_results == True:
        print(pred_tokens_dict)

    return percent_right

# Numerals

In [24]:
def generate_prompts_list():
    prompts_list = []
    for i in range(1, 98):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+1),
            'S3': str(i+2),
            'S4': str(i+3),
            'text': f"{i} {i+1} {i+2} {i+3}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list()

In [25]:
dataset = Dataset(prompts_list, model.tokenizer, S1_is_first=True)

In [26]:
get_next_scores(model, 9, 1, dataset, task="numerals")

Next circuit for head 9.1: Top 5 accuracy: 87.37113402061856%
{'1': ([' one', ' needed', ' preferred', ' single', '2'], '2', 'yes'), '2': ([' third', ' fourth', 'third', '3', ' fifth'], '3', 'yes'), '3': ([' fourth', ' third', ' fifth', 'Fourth', 'fourth'], '4', 'no'), '4': ([' fifth', ' sixth', ' seventh', 'fifth', 'five'], '5', 'no'), '5': ([' sixth', ' seventh', ' fifth', '6', ' eighth'], '6', 'yes'), '6': ([' seventh', ' Seventh', ' sixth', '7', ' eighth'], '7', 'yes'), '7': ([' seventh', ' eighth', ' ninth', ' VIII', ' Seventh'], '8', 'no'), '8': ([' ninth', ' eighth', 'ighth', ' seventh', '9'], '9', 'yes'), '9': ([' ninth', ' seventh', ' tenth', ' eighth', ' sixth'], '10', 'no'), '10': ([' tenth', ' seventh', ' eighth', ' ninth', ' sixth'], '11', 'no'), '11': ([' eighth', ' seventh', ' specific', ' 12', '12'], '12', 'yes'), '12': ([' seventh', ' eighth', '13', ' specific', '14'], '13', 'yes'), '13': ([' seventh', '14', ' 14', ' eighth', 'Only'], '14', 'yes'), '14': ([' eighth', '

87.37113402061856

In [28]:
get_copy_scores(model, 9, 1, dataset)

Copy circuit for head 9.1: Top 5 accuracy: 59.27835051546392%
{'1': ([' one', ' needed', ' preferred', ' single', '2'], 'no'), '2': ([' third', ' fourth', 'third', '3', ' fifth'], 'no'), '3': ([' fourth', ' third', ' fifth', 'Fourth', 'fourth'], 'no'), '4': ([' fifth', ' sixth', ' seventh', 'fifth', 'five'], 'no'), '5': ([' sixth', ' seventh', ' fifth', '6', ' eighth'], 'no'), '6': ([' seventh', ' Seventh', ' sixth', '7', ' eighth'], 'no'), '7': ([' seventh', ' eighth', ' ninth', ' VIII', ' Seventh'], 'no'), '8': ([' ninth', ' eighth', 'ighth', ' seventh', '9'], 'no'), '9': ([' ninth', ' seventh', ' tenth', ' eighth', ' sixth'], 'no'), '10': ([' tenth', ' seventh', ' eighth', ' ninth', ' sixth'], 'no'), '11': ([' eighth', ' seventh', ' specific', ' 12', '12'], 'no'), '12': ([' seventh', ' eighth', '13', ' specific', '14'], 'no'), '13': ([' seventh', '14', ' 14', ' eighth', 'Only'], 'no'), '14': ([' eighth', ' seventh', ' fifth', ' maximum', ' ninth'], 'no'), '15': ([' seventh', ' eight

59.27835051546392

In [33]:
# %%capture
all_copy_scores = []
all_heads = [(layer, head) for layer in range(12) for head in range(12)]
for index, (layer, head) in enumerate(all_heads):
    all_copy_scores.append(get_copy_scores(model, layer, head, dataset, print_all_results=False))
    # all_copy_scores.append( (layer, head, get_copy_scores(model, layer, head, dataset, print_all_results=False)) )

Copy circuit for head 1.7: Top 5 accuracy: 2.0618556701030926%
Copy circuit for head 2.1: Top 5 accuracy: 1.5463917525773196%
Copy circuit for head 2.4: Top 5 accuracy: 0.25773195876288657%
Copy circuit for head 3.5: Top 5 accuracy: 1.2886597938144329%
Copy circuit for head 4.8: Top 5 accuracy: 0.25773195876288657%
Copy circuit for head 5.0: Top 5 accuracy: 11.34020618556701%
Copy circuit for head 5.1: Top 5 accuracy: 48.71134020618557%
Copy circuit for head 5.5: Top 5 accuracy: 0.25773195876288657%
Copy circuit for head 6.1: Top 5 accuracy: 1.804123711340206%
Copy circuit for head 6.4: Top 5 accuracy: 0.5154639175257731%
Copy circuit for head 6.9: Top 5 accuracy: 83.76288659793815%
Copy circuit for head 6.10: Top 5 accuracy: 1.0309278350515463%
Copy circuit for head 7.1: Top 5 accuracy: 1.0309278350515463%
Copy circuit for head 7.2: Top 5 accuracy: 79.12371134020619%
Copy circuit for head 7.5: Top 5 accuracy: 0.7731958762886598%
Copy circuit for head 7.7: Top 5 accuracy: 2.31958762886

In [34]:
sum(all_copy_scores)/len(all_copy_scores)

5.725587056128294

In [31]:
# %%capture
all_next_scores = []
all_heads = [(layer, head) for layer in range(12) for head in range(12)]
for index, (layer, head) in enumerate(all_heads):
    all_next_scores.append(get_next_scores(model, layer, head, dataset, task="numerals", print_all_results=False))
    # all_next_scores.append( (layer, head, get_next_scores(model, layer, head, dataset, task="numerals", print_tokens=False)) )

Next circuit for head 1.7: Top 5 accuracy: 0.7731958762886598%
Next circuit for head 2.1: Top 5 accuracy: 0.25773195876288657%
Next circuit for head 3.5: Top 5 accuracy: 2.3195876288659796%
Next circuit for head 4.9: Top 5 accuracy: 0.25773195876288657%
Next circuit for head 5.0: Top 5 accuracy: 7.731958762886598%
Next circuit for head 5.1: Top 5 accuracy: 2.0618556701030926%
Next circuit for head 5.5: Top 5 accuracy: 0.25773195876288657%
Next circuit for head 6.1: Top 5 accuracy: 14.948453608247423%
Next circuit for head 6.9: Top 5 accuracy: 57.73195876288659%
Next circuit for head 7.2: Top 5 accuracy: 31.185567010309278%
Next circuit for head 7.7: Top 5 accuracy: 1.5463917525773196%
Next circuit for head 7.8: Top 5 accuracy: 1.2886597938144329%
Next circuit for head 7.10: Top 5 accuracy: 48.71134020618557%
Next circuit for head 7.11: Top 5 accuracy: 9.278350515463918%
Next circuit for head 8.0: Top 5 accuracy: 1.0309278350515463%
Next circuit for head 8.1: Top 5 accuracy: 33.50515463

In [32]:
sum(all_next_scores)/len(all_next_scores)

3.2932416953035517

# Numwords

In [ ]:
def generate_prompts_list(x ,y):
    words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': words[i],
            'S2': words[i+1],
            'S3': words[i+2],
            'S4': words[i+3],
            'text': f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(0, 8)
dataset = Dataset(prompts_list, model.tokenizer, S1_is_first=True)

In [ ]:
all_next_scores = []
all_heads = [(layer, head) for layer in range(12) for head in range(12)]
for index, (layer, head) in enumerate(all_heads):
    all_next_scores.append(get_next_scores(model, layer, head, dataset, task="numwords", print_all_results=False))

Next circuit for head 1.7: Top 5 accuracy: 3.125%
Next circuit for head 2.1: Top 5 accuracy: 3.125%
Next circuit for head 4.8: Top 5 accuracy: 3.125%
Next circuit for head 5.0: Top 5 accuracy: 9.375%
Next circuit for head 6.1: Top 5 accuracy: 75.0%
Next circuit for head 7.2: Top 5 accuracy: 21.875%
Next circuit for head 7.7: Top 5 accuracy: 18.75%
Next circuit for head 7.11: Top 5 accuracy: 6.25%
Next circuit for head 8.1: Top 5 accuracy: 43.75%
Next circuit for head 8.8: Top 5 accuracy: 56.25%
Next circuit for head 9.1: Top 5 accuracy: 90.625%
Next circuit for head 9.7: Top 5 accuracy: 9.375%
Next circuit for head 10.2: Top 5 accuracy: 43.75%
Next circuit for head 11.4: Top 5 accuracy: 43.75%


In [ ]:
sum(all_next_scores)/len(all_next_scores)

2.9730902777777777

# Months

In [35]:
def generate_prompts_list():
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
    prompts_list = []
    for i in range(0, 8):
        prompt_dict = {
            'S1': months[i],
            'S2': months[i+1],
            'S3': months[i+2],
            'S4': months[i+3],
            'text': f"{months[i]} {months[i+1]} {months[i+2]} {months[i+3]}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list()
dataset = Dataset(prompts_list, model.tokenizer, S1_is_first=True)

In [36]:
all_copy_scores = []
all_heads = [(layer, head) for layer in range(12) for head in range(12)]
for index, (layer, head) in enumerate(all_heads):
    all_copy_scores.append(get_copy_scores(model, layer, head, dataset, print_all_results=False))

Copy circuit for head 5.0: Top 5 accuracy: 34.375%
Copy circuit for head 5.1: Top 5 accuracy: 31.25%
Copy circuit for head 5.4: Top 5 accuracy: 6.25%
Copy circuit for head 5.7: Top 5 accuracy: 3.125%
Copy circuit for head 6.0: Top 5 accuracy: 3.125%
Copy circuit for head 6.1: Top 5 accuracy: 3.125%
Copy circuit for head 6.9: Top 5 accuracy: 81.25%
Copy circuit for head 6.10: Top 5 accuracy: 6.25%
Copy circuit for head 7.2: Top 5 accuracy: 75.0%
Copy circuit for head 7.5: Top 5 accuracy: 3.125%
Copy circuit for head 7.7: Top 5 accuracy: 9.375%
Copy circuit for head 7.8: Top 5 accuracy: 59.375%
Copy circuit for head 7.10: Top 5 accuracy: 71.875%
Copy circuit for head 7.11: Top 5 accuracy: 71.875%
Copy circuit for head 8.1: Top 5 accuracy: 75.0%
Copy circuit for head 8.6: Top 5 accuracy: 9.375%
Copy circuit for head 8.7: Top 5 accuracy: 3.125%
Copy circuit for head 8.8: Top 5 accuracy: 6.25%
Copy circuit for head 8.11: Top 5 accuracy: 65.625%
Copy circuit for head 9.3: Top 5 accuracy: 28.

In [37]:
sum(all_copy_scores)/len(all_copy_scores)

7.400173611111111

In [39]:
all_next_scores = []
all_heads = [(layer, head) for layer in range(12) for head in range(12)]
for index, (layer, head) in enumerate(all_heads):
    all_next_scores.append(get_next_scores(model, layer, head, dataset, task="months", print_all_results=False))

Next circuit for head 9.1: Top 5 accuracy: 31.25%


In [40]:
sum(all_next_scores)/len(all_next_scores)

0.2170138888888889

In [41]:
get_next_scores(model, 9, 1, dataset, task="months")

Next circuit for head 9.1: Top 5 accuracy: 31.25%
{'January': ([' once', 'once', ' occasional', ' seventh', '296'], 'second', 'no'), 'February': ([' third', ' fourth', ' once', ' seventh', ' fifth'], 'third', 'yes'), 'March': ([' fifth', ' seventh', ' fourth', ' sixth', ' third'], 'fourth', 'yes'), 'April': ([' seventh', ' fifth', ' sixth', 'ISC', ' five'], 'fifth', 'yes'), 'May': ([' seventh', ' sixth', 'ISC', ' once', ' 157'], 'sixth', 'yes'), 'June': ([' seventh', ' third', ' seven', ' sixth', 'seven'], 'seventh', 'yes'), 'July': ([' seventh', ' eighth', 'seven', ' once', ' seven'], 'eighth', 'yes'), 'August': (['ighth', ' eighth', ' ninth', ' final', ' occasional'], 'ninth', 'yes'), 'September': ([' 120', 'ure', 'Loading', '�', ' Nguyen'], 'tenth', 'no'), 'October': ([' Nur', 'undrum', '�', '�', ' 121'], 'eleventh', 'no'), 'November': (['oor', 'ة', ' 122', 'Enlarge', ' Nur'], 'twelfth', 'no')}


31.25